# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
"""
Change directory to where this file is located
"""
#%cd 'COPY&PASTE FILE DIRECTORY HERE'
%cd '/content/drive/MyDrive/Colab Notebooks/'

/content/drive/MyDrive/Colab Notebooks


# Import Modules

In [3]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [4]:
def leaky_relu(z):
    """
    Implement the leaky ReLU activation function.
    The method takes the input z and returns the output of the function.
    
    Set the value of alpha for the leaky ReLU funtion to 0.01.
    Question (a)

    """
    #### YOUR CODE #####
    return np.maximum(0.01*z, z)
    #####################

def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    exp_X = np.exp(X)
    sum_exp_X = np.sum(exp_X, axis=1, keepdims=True)

    return exp_X/sum_exp_X
    #####################

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

#2-Layer Neural Network

In [5]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with He Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/he-initialization for He initialization 
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        ##### YOUR CODE #####
        params["W1"] = np.random.randn(input_dim, num_hiddens)*np.sqrt(2/input_dim)
        params["b1"] = np.zeros((1,num_hiddens))  
        params["W2"] = np.random.randn(num_hiddens, num_classes)*np.sqrt(2/num_hiddens)
        params["b2"] = np.zeros((1,num_classes))
        #####################
        
        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(leaky_relu(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        ##### YOUR CODE #####
        W1 = self.params["W1"]
        b1 = self.params["b1"]
        Z1 = np.matmul(X, W1)+b1
        
        A1 = leaky_relu(Z1)
        W2 = self.params["W2"]
        b2 = self.params["b2"]
        Z2 = np.matmul(A1, W2)+ b2
        y = softmax(Z2)
        ff_dict["Z1"]= Z1
        ff_dict["A1"] = A1
        ff_dict["Z2"]= Z2
        ff_dict["y"] = y
        ff_dict["W2"]=W2
        #####################        
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        ##### YOUR CODE #####
        m = X.shape[0]
        dZ2 = ff_dict["y"] - Y
        db2 = np.sum(dZ2, axis=0, keepdims=True)
        dW2 = np.dot(ff_dict["A1"].T,dZ2) 
        dA1 = np.dot(dZ2, ff_dict["W2"].T)

        #leaky relu = dZ1계산
        dx = np.ones_like(dA1)
        dx[ff_dict["Z1"]<0] = 0.01 
        dZ1 = dA1 * dx 

        db1 = np.sum(dZ1, axis=0, keepdims=True)
        dW1 = np.matmul(X.T, dZ1)

        grads["dW1"] = dW1 
        grads["db1"] = db1  
        grads["dW2"] = dW2
        grads["db2"] = db2
        #####################
        
        return grads

    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        epsilon = 1e-10
        Y_hat = np.clip(Y_hat, epsilon, 1 - epsilon)
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

#Load MNIST

In [6]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [7]:
### 
# Question (e)
# Tune the hyperparameters with validation data, 
# and print the results by running the lines below.
###

In [17]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=64, num_classes=10)

In [18]:
# train the model
lr, n_epochs, batch_size = 0.01, 10, 128

model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.308 0.912, valid loss/acc: 0.316 0.910
epoch 01 - train loss/acc: 0.244 0.931, valid loss/acc: 0.255 0.927
epoch 02 - train loss/acc: 0.209 0.942, valid loss/acc: 0.222 0.938
epoch 03 - train loss/acc: 0.188 0.946, valid loss/acc: 0.205 0.941
epoch 04 - train loss/acc: 0.162 0.954, valid loss/acc: 0.177 0.950
epoch 05 - train loss/acc: 0.149 0.958, valid loss/acc: 0.165 0.953
epoch 06 - train loss/acc: 0.136 0.961, valid loss/acc: 0.156 0.954
epoch 07 - train loss/acc: 0.126 0.965, valid loss/acc: 0.146 0.958
epoch 08 - train loss/acc: 0.117 0.967, valid loss/acc: 0.138 0.959
epoch 09 - train loss/acc: 0.107 0.970, valid loss/acc: 0.132 0.962


In [19]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.122, acc = 0.963


In [ ]:
#결과분석
# learning rate을 0.01로 조정하고 epoch 수는 10, batch_size는 128로 설정하였을때
# final acc가 0.963d으로 높게 나옴
# learnig rate이 2일 경우에는 학습이 제대로 되지 않기 때문에 낮게 조정해야함.
